In [219]:
#%pip install mido
#%pip install python-rtmidi
import keras_tuner as kt
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from mido import MidiFile, MidiTrack, Message

In [232]:
def preprocessing(data):
    return tf.reshape(tf.where(data == 0, data, data - min_note + 1), [-1])

def get_data(data):
    return [pd.read_csv(file).values.tolist() for file in data]

def target(data):
    X = data[:, :-1]
    Y = data[:, 1:]
    return X,Y

def save(data, name):
    music_file = MidiFile()
    track = MidiTrack()
    music_file.tracks.append(track)
    for sequence in train_data:
        for chord in sequence:
            for note in chord:
                track.append(Message('note_on', note =note, velocity = 64, time = 0))
            track.append(Message('note_off', note = chord[0], velocity = 64, time = 480))
            for note in chord:
                track.append(Message('note_off', note = note, velocity = 64, time = 0))
    output = f"{name}.mid"
    music_file.save(output)

def create_dataset(chorales, batch_size=32, shuffle=None, window_size=32, window_shift=16):
    def batchW(window):
        return window.batch(window_size + 2)

    def toW(chorale):
        dataset = tf.data.Dataset.from_tensor_slices(chorale)
        dataset = dataset.window(window_size + 2, window_shift, drop_remainder=True)
        return dataset.flat_map(batchW)

    chorales = tf.ragged.constant(chorales, ragged_rank=1)
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
    dataset = dataset.flat_map(toW).map(preprocessing)
    dataset = dataset.cache()
    if shuffle:
        dataset = dataset.shuffle(shuffle)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(target)
    return dataset.prefetch(tf.data.AUTOTUNE)

In [201]:
file_train = Path('train')
file_valid = Path('valid')
file_test = Path('test')

In [202]:
train = sorted(file_train.glob('chorale_*.csv'))
valid = sorted(file_valid.glob('chorale_*.csv'))
test = sorted(file_test.glob('chorale_*.csv'))

In [203]:
print(train[:5])
print("--------------")
print(valid[:5])
print("--------------")
print(test[:5])

[WindowsPath('train/chorale_000.csv'), WindowsPath('train/chorale_001.csv'), WindowsPath('train/chorale_002.csv'), WindowsPath('train/chorale_003.csv'), WindowsPath('train/chorale_004.csv')]
--------------
[WindowsPath('valid/chorale_229.csv'), WindowsPath('valid/chorale_230.csv'), WindowsPath('valid/chorale_231.csv'), WindowsPath('valid/chorale_232.csv'), WindowsPath('valid/chorale_233.csv')]
--------------
[WindowsPath('test/chorale_305.csv'), WindowsPath('test/chorale_306.csv'), WindowsPath('test/chorale_307.csv'), WindowsPath('test/chorale_308.csv'), WindowsPath('test/chorale_309.csv')]


In [204]:
train_data = get_data(train)
valid_data = get_data(valid)
test_data = get_data(test)
notes = set()
for i in (train_data, valid_data, test_data):
    for j in i:
        for k in j:
            notes |= set(k)

n_notes = len(notes)
min_note = min(notes - {0})
max_note = max(notes)

assert min_note == 36
assert max_note == 81

In [205]:
print(train_data[:1])
print("--------------")
print(valid_data[:1])
print("--------------")
print(test_data[:1])

[[[74, 70, 65, 58], [74, 70, 65, 58], [74, 70, 65, 58], [74, 70, 65, 58], [75, 70, 58, 55], [75, 70, 58, 55], [75, 70, 60, 55], [75, 70, 60, 55], [77, 69, 62, 50], [77, 69, 62, 50], [77, 69, 62, 50], [77, 69, 62, 50], [77, 70, 62, 55], [77, 70, 62, 55], [77, 69, 62, 55], [77, 69, 62, 55], [75, 67, 63, 48], [75, 67, 63, 48], [75, 69, 63, 48], [75, 69, 63, 48], [74, 70, 65, 46], [74, 70, 65, 46], [74, 70, 65, 46], [74, 70, 65, 46], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [74, 70, 65, 46], [74, 70, 65, 46], [74, 70, 65, 46], [74, 70, 65, 46], [75, 69, 63, 48], [75, 69, 63, 48], [75, 67, 63, 48], [75, 67, 63, 48], [77, 65, 62, 50], [77, 65, 62, 50], [77, 65, 60, 50], [77, 65, 60, 50], [74, 67, 58, 55], [74, 67, 58, 55], [74, 67, 58, 53], [74, 67, 58, 53], [72, 67, 58, 51], [72, 67, 58, 51], [72, 67, 58, 51], [72, 67, 58, 51], [72, 65, 57, 53], [72, 65, 57, 53], [72, 65, 57, 53], [72, 65,

In [206]:
music_file = MidiFile()
track = MidiTrack()
music_file.tracks.append(track)

In [207]:
for sequence in train_data:
    for chord in sequence:
        for note in chord:
            track.append(Message('note_on', note =note, velocity = 64, time = 0))
        track.append(Message('note_off', note = chord[0], velocity = 64, time = 480))
        for note in chord:
            track.append(Message('note_off', note = note, velocity = 64, time = 0))

In [208]:
output = 'music.mid'
music_file.save(output)

In [209]:
print(tf.ragged.constant(train_data, ragged_rank=1)[:1])

<tf.RaggedTensor [[[74, 70, 65, 58],
  [74, 70, 65, 58],
  [74, 70, 65, 58],
  [74, 70, 65, 58],
  [75, 70, 58, 55],
  [75, 70, 58, 55],
  [75, 70, 60, 55],
  [75, 70, 60, 55],
  [77, 69, 62, 50],
  [77, 69, 62, 50],
  [77, 69, 62, 50],
  [77, 69, 62, 50],
  [77, 70, 62, 55],
  [77, 70, 62, 55],
  [77, 69, 62, 55],
  [77, 69, 62, 55],
  [75, 67, 63, 48],
  [75, 67, 63, 48],
  [75, 69, 63, 48],
  [75, 69, 63, 48],
  [74, 70, 65, 46],
  [74, 70, 65, 46],
  [74, 70, 65, 46],
  [74, 70, 65, 46],
  [72, 69, 65, 53],
  [72, 69, 65, 53],
  [72, 69, 65, 53],
  [72, 69, 65, 53],
  [72, 69, 65, 53],
  [72, 69, 65, 53],
  [72, 69, 65, 53],
  [72, 69, 65, 53],
  [74, 70, 65, 46],
  [74, 70, 65, 46],
  [74, 70, 65, 46],
  [74, 70, 65, 46],
  [75, 69, 63, 48],
  [75, 69, 63, 48],
  [75, 67, 63, 48],
  [75, 67, 63, 48],
  [77, 65, 62, 50],
  [77, 65, 62, 50],
  [77, 65, 60, 50],
  [77, 65, 60, 50],
  [74, 67, 58, 55],
  [74, 67, 58, 55],
  [74, 67, 58, 53],
  [74, 67, 58, 53],
  [72, 67, 58, 51],
  [

In [210]:
h = tf.data.Dataset.from_tensor_slices(tf.ragged.constant(train_data, ragged_rank=1))
for i in h.take(1):
    print(i)

tf.Tensor(
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]
 [77 70 62 55]
 [77 70 62 55]
 [77 69 62 55]
 [77 69 62 55]
 [75 67 63 48]
 [75 67 63 48]
 [75 69 63 48]
 [75 69 63 48]
 [74 70 65 46]
 [74 70 65 46]
 [74 70 65 46]
 [74 70 65 46]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [74 70 65 46]
 [74 70 65 46]
 [74 70 65 46]
 [74 70 65 46]
 [75 69 63 48]
 [75 69 63 48]
 [75 67 63 48]
 [75 67 63 48]
 [77 65 62 50]
 [77 65 62 50]
 [77 65 60 50]
 [77 65 60 50]
 [74 67 58 55]
 [74 67 58 55]
 [74 67 58 53]
 [74 67 58 53]
 [72 67 58 51]
 [72 67 58 51]
 [72 67 58 51]
 [72 67 58 51]
 [72 65 57 53]
 [72 65 57 53]
 [72 65 57 53]
 [72 65 57 53]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [72 69 65 53]
 [72 69 65 53]

In [211]:
for i in h.batch(2):
    print(i.numpy())

[array([[74, 70, 65, 58],
        [74, 70, 65, 58],
        [74, 70, 65, 58],
        [74, 70, 65, 58],
        [75, 70, 58, 55],
        [75, 70, 58, 55],
        [75, 70, 60, 55],
        [75, 70, 60, 55],
        [77, 69, 62, 50],
        [77, 69, 62, 50],
        [77, 69, 62, 50],
        [77, 69, 62, 50],
        [77, 70, 62, 55],
        [77, 70, 62, 55],
        [77, 69, 62, 55],
        [77, 69, 62, 55],
        [75, 67, 63, 48],
        [75, 67, 63, 48],
        [75, 69, 63, 48],
        [75, 69, 63, 48],
        [74, 70, 65, 46],
        [74, 70, 65, 46],
        [74, 70, 65, 46],
        [74, 70, 65, 46],
        [72, 69, 65, 53],
        [72, 69, 65, 53],
        [72, 69, 65, 53],
        [72, 69, 65, 53],
        [72, 69, 65, 53],
        [72, 69, 65, 53],
        [72, 69, 65, 53],
        [72, 69, 65, 53],
        [74, 70, 65, 46],
        [74, 70, 65, 46],
        [74, 70, 65, 46],
        [74, 70, 65, 46],
        [75, 69, 63, 48],
        [75, 69, 63, 48],
        [75,

In [212]:
train_set = create_dataset(train_data, shuffle=1000)
valid_set = create_dataset(valid_data)
test_set = create_dataset(test_data)

In [213]:
num_batches = sum(1 for _ in train_set)
print(f"Number of batches: {num_batches}")
for i in train_set:
    print(i)

Number of batches: 98
(<tf.Tensor: shape=(32, 135), dtype=int32, numpy=
array([[36, 27, 21, ..., 29, 24, 21],
       [40, 32, 25, ..., 32, 28, 23],
       [40, 32, 23, ..., 39, 35, 30],
       ...,
       [27, 22, 18, ..., 37, 34, 29],
       [34, 27, 19, ..., 37, 32, 25],
       [39, 34, 22, ..., 39, 34, 27]])>, <tf.Tensor: shape=(32, 135), dtype=int32, numpy=
array([[27, 21, 12, ..., 24, 21, 17],
       [32, 25, 13, ..., 28, 23, 16],
       [32, 23, 16, ..., 35, 30, 21],
       ...,
       [22, 18, 15, ..., 34, 29, 22],
       [27, 19, 15, ..., 32, 25, 22],
       [34, 22, 19, ..., 34, 27, 19]])>)
(<tf.Tensor: shape=(32, 135), dtype=int32, numpy=
array([[30, 25, 22, ..., 37, 32, 20],
       [37, 29, 22, ..., 29, 32, 25],
       [30, 27, 22, ..., 34, 29, 25],
       ...,
       [44, 28, 23, ..., 40, 32, 25],
       [27, 22, 18, ..., 34, 27, 22],
       [37, 34, 17, ..., 27, 27, 19]])>, <tf.Tensor: shape=(32, 135), dtype=int32, numpy=
array([[25, 22,  6, ..., 32, 20, 16],
       [29, 2

In [228]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Embedding(input_dim=n_notes, output_dim=hp.Int('embedding_dim', min_value=8, max_value=128, step=8),input_shape=[None]),
            tf.keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LSTM(512, return_sequences=True),
            tf.keras.layers.Dense(n_notes, activation="softmax")
        ])
        
       
        model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Nadam(), metrics=["accuracy"])
        return model


tuner = kt.Hyperband(MyHyperModel(),objective='val_accuracy',max_epochs=10,hyperband_iterations=2,directory='my_dir',project_name='project_name')


tuner.search(train_set.repeat(), epochs=5, validation_data=valid_set, steps_per_epoch = 98)



Trial 22 Complete [00h 17m 46s]
val_accuracy: 0.38042280077934265

Best val_accuracy So Far: 0.8049192428588867
Total elapsed time: 05h 28m 00s


In [233]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 128)      │         6,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, None, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, None, 48)       │         3,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 48)       │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, None, 64)       │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, None, 96)       │        12,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, None, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, None, 32)       │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, None, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, None, 48)       │         3,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, None, 48)       │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, None, 64)       │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, None, 96)       │        12,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, None, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, None, 32)       │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, None, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, None, 48)       │         3,120 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,365,951 (5.21 MB)

 Trainable params: 1,364,511 (5.21 MB)

 Non-trainable params: 1,440 (5.62 KB)

In [234]:

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("best_model.keras",monitor="val_accuracy",save_best_only=True,verbose=1, mode='max')

early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1, mode='max')

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=1e-6, mode='max')

In [235]:

best_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Nadam(),
              metrics=["accuracy"])
best_model.fit(train_set.repeat(20), epochs=20, validation_data=valid_set, callbacks=[checkpoint_callback, early_stopping_callback, reduce_lr_callback], steps_per_epoch = 98)

Epoch 1/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8766 - loss: 0.3910
Epoch 1: val_accuracy improved from -inf to 0.80376, saving model to best_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 402s 4s/step - accuracy: 0.8766 - loss: 0.3908 - val_accuracy: 0.8038 - val_loss: 0.6732 - learning_rate: 0.0010
Epoch 2/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8890 - loss: 0.3494
Epoch 2: val_accuracy did not improve from 0.80376
98/98 ━━━━━━━━━━━━━━━━━━━━ 374s 4s/step - accuracy: 0.8891 - loss: 0.3493 - val_accuracy: 0.7988 - val_loss: 0.6956 - learning_rate: 0.0010
Epoch 3/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8959 - loss: 0.3281
Epoch 3: val_accuracy did not improve from 0.80376
98/98 ━━━━━━━━━━━━━━━━━━━━ 382s 4s/step - accuracy: 0.8959 - loss: 0.3280 - val_accuracy: 0.8034 - val_loss: 0.6980 - learning_rate: 0.0010
Epoch 4/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9044 - loss: 0.3008
Epoch 4: val_accuracy did not improve from 0.80376

Epoch 4:

In [236]:
best_model.evaluate(test_set)

34/34 ━━━━━━━━━━━━━━━━━━━━ 39s 731ms/step - accuracy: 0.8012 - loss: 0.7462


[0.7243715524673462, 0.8052414655685425]

In [237]:
def generation(model, seed_chords, length, risk = 1):
    music = preprocessing(tf.constant(seed_chords, dtype=tf.int64))
    music = tf.reshape(music, [1, -1])
    for chord in range(length):
        for note in range(4):
            next_note_probas = model.predict(music)[0, -1:]
            rescaled = tf.math.log(next_note_probas) / risk
            next_note = tf.random.categorical(rescaled, num_samples=1)
            music = tf.concat([music, next_note], axis=1)
    music = tf.where(music == 0, music, music + min_note - 1)
    return tf.reshape(music, shape=[-1, 4])

In [238]:
n = len(test_data)
seed_chords = test_data[np.random.randint(n)][:8]
music = generation(best_model, seed_chords, 56)
save(music, 'music1')

1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 933ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 830ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 514ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms

In [239]:
music = generation(best_model, test_data[np.random.randint(n)][:8], 56, risk = 2)
save(music, 'music2')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━

In [240]:
music = generation(best_model, test_data[np.random.randint(n)][:12], 56, risk = 1.5)
save(music, 'music3')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━

In [241]:
music = generation(best_model, test_data[np.random.randint(n)][:32], 56, risk = 3)
save(music, 'music4')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

In [242]:
music = generation(best_model, test_data[np.random.randint(n)][:64], 56, risk = 4)
save(music, 'music5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 682ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 526ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 